In [51]:
# Gestion des chemins et des modules
import sys
import os
import importlib

# Telechargement et parsing HTML
import requests
import urllib.request
from bs4 import BeautifulSoup

# Gestion des fichiers
from zipfile import ZipFile
from PyPDF2 import PdfReader

# Manipulation des bases de donnees
import sqlite3
import gestion_database as gd

# Traitement du texte et des chaines
import re
import unicodedata
import copy
import Levenshtein
from Levenshtein import distance as levenshtein_distance

# Machine Learning et Clustering
import numpy as np
import random
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Rechargement de modules (nécessaire pour gd)
importlib.reload(gd)


<module 'gestion_database' from 'j:\\01 Fichier\\05 Code\\99 - Autre\\02 Sujetdex\\gestion_database.py'>

In [46]:
def jaccard_similarity(word1, word2, n=2):
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(n, n), binary=True)
    X = vectorizer.fit_transform([word1, word2]).toarray()
    return jaccard_score(X[0], X[1])

In [4]:
def extract_links_from_page(url):
    """
    Extracts all hyperlinks from a webpage.

    Parameters:
        url (str): The URL of the webpage to scrape.

    Returns:
        list: A list of URLs found on the page.
    """
    try:
        # Make an HTTP request to fetch the page content
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return []

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <a> tags with an href attribute
    links = [requests.compat.urljoin(url, link['href']) for link in soup.find_all('a', href=True)]

    return links

visited_links = {}

def fetch_files_from_page(url, output_folder="downloads", file_extensions=("pdf", "doc", "docx"), filter_function=None):
    """
    Fetches all files with specified extensions from a given URL and saves them locally.

    Parameters:
        url (str): The URL of the webpage to scrape.
        output_folder (str): The folder to save the downloaded files. Defaults to "downloads".
        file_extensions (tuple): File extensions to download. Defaults to ("pdf", "doc", "docx").
        filter_function (callable): A function that takes a file name as input and returns True if the file should be downloaded, False otherwise. Defaults to None.

    Returns:
        list: List of downloaded file paths.
    """
    # Dictionary to track visited links and avoid duplicates
    

    # Get all links from the page
    all_links = extract_links_from_page(url)
    file_links = []

    for file_url in all_links:
        # Skip if the link has already been processed
        if file_url in visited_links:
            continue

        # Mark the link as visited
        visited_links[file_url] = True

        # Check file extension
        if file_url.split('.')[-1].lower() in file_extensions:
            file_links.append(file_url)

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    downloaded_files = []

    # Download each file
    for file_url in file_links:
        try:
            file_response = requests.get(file_url, stream=True)
            file_response.raise_for_status()

            # Extract the file name
            file_name = os.path.basename(file_url)

            # Apply the filter function if provided
            if filter_function and not filter_function(file_name):
                continue

            output_path = os.path.join(output_folder, file_name)

            # Save the file locally
            with open(output_path, 'wb') as f:
                for chunk in file_response.iter_content(chunk_size=8192):
                    f.write(chunk)

            downloaded_files.append(output_path)
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {file_url}: {e}")

    return downloaded_files

In [5]:
def link_all_files(folder: str) -> list[str]:
    """Fonction qui renvoie tout les fichiers pdf d'un dossier et de ses sous-dossiers
    
    param folder: le chemin du dossier à explorer
    type folder: str
    return: la liste des fichiers pdf
    rtype: list[str]
    """
    linked_files = []

    for root, _, files in os.walk(folder):
        for file in files:
            file_path = os.path.join(root, file)
            linked_files.append(file_path)

    return linked_files

# Fonction en lien avec le PDF

In [6]:
def extract_text_from_pdf(file_path:str) -> list[str]:
    """Fonction qui renvoie le texte du PDF 

    :param file_path: lien du fichier
    :type file_path: str
    """
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        pages = pdf_reader.pages
        resultats = [page.extract_text() for page in pages]
        return resultats

In [7]:
def extract_text_of_page_from_pdf(file_path:str, i: int) -> str:
    """Fonction qui renvoie la page spécifiée d'un document pdf.

    :param file_path: lien du fichier
    :type file_path: str
    :param i: numero de la page a extraire (commence avec la page 1)
    :type i: int
    :return: Text de la page
    :rtype: str
    """    
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        pages = pdf_reader.pages
        try:
            resultats = pages[i-1].extract_text()
            return resultats
        except IndexError:
            raise Exception("La page demandée n'existe pas")
        
    

In [8]:
def extract_first_page_from_pdf(file_path:str) -> str:
    return extract_text_of_page_from_pdf(file_path, 1)

# Centrale

In [8]:
baselink = "https://www.concours-centrale-supelec.fr/CentraleSupelec"
annee = [i for i in range(1998,2025)]
filieres = ["MP","PC","MPI","PSI","TSI"]
sujet = ["","sujets"]

In [9]:
def get_filieres():
    dossier = f"data\\centrale"
    liste_filieres = []
    for filiere in os.listdir(dossier):
        liste_filieres.append(filiere)
    return liste_filieres

filieres = get_filieres()

In [10]:
def supprimer_fichiers_contenant_recursivement(repertoire: str, chaine_a_chercher: str) -> None:
    """Supprime tous les fichiers dans un répertoire et ses sous-répertoires
    dont le nom contient une chaîne de caractères spécifique.

    :param repertoire: Chemin du répertoire où commencer la recherche
    :type repertoire: str
    :param chaine_a_chercher: Chaîne de caractères à chercher dans les noms de fichiers
    :type chaine_a_chercher: str
    :return: Cette fonction ne retourne rien
    :rtype: None
    """
    for root, dirs, files in os.walk(repertoire):
        for file in files:
            if chaine_a_chercher in file:
                chemin_complet = os.path.join(root, file)
                os.remove(chemin_complet)
                print(f"Fichier supprimé : {chemin_complet}")


In [11]:
"""for f in filiere:
    for a in annee:
        for sujets in sujet:
            url = f"{baselink}/{a}/{f}/{sujets}"
            fetch_files_from_page(url, output_folder=f"data/centrale/{f}/{a}", filter_function=filtre)"""

'for f in filiere:\n    for a in annee:\n        for sujets in sujet:\n            url = f"{baselink}/{a}/{f}/{sujets}"\n            fetch_files_from_page(url, output_folder=f"data/centrale/{f}/{a}", filter_function=filtre)'

In [12]:
def supprimer_fichiers_contenant_recursivement(repertoire: str, chaine_a_chercher: str) -> None:
    """Supprime tous les fichiers dans un répertoire et ses sous-répertoires
    dont le nom contient une chaîne de caractères spécifique.

    :param repertoire: Chemin du répertoire où commencer la recherche
    :type repertoire: str
    :param chaine_a_chercher: Chaîne de caractères à rechercher dans les noms de fichiers
    :type chaine_a_chercher: str
    :return: Cette fonction ne retourne rien
    :rtype: None
    """
    for root, dirs, files in os.walk(repertoire):
        for file in files:
            if chaine_a_chercher in file:
                chemin_complet = os.path.join(root, file)
                os.remove(chemin_complet)
                print(f"Fichier supprimé : {chemin_complet}")


In [13]:
"""supprimer_fichiers_contenant_recursivement(repertoire= "data", chaine_a_chercher = "TSI")"""

'supprimer_fichiers_contenant_recursivement(repertoire= "data", chaine_a_chercher = "TSI")'

In [14]:
def extraire_titres_et_questions(fichier_pdf: str) -> list:
    """Extrait les titres et numéros de questions d'un fichier PDF.

    :param fichier_pdf: Chemin vers le fichier PDF
    :type fichier_pdf: str
    :return: Liste des numéros de questions extraites du fichier PDF
    :rtype: list
    """
    with open(fichier_pdf, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        pages = pdf_reader.pages

        resultats = []
        for page in pages:
            texte_page = page.extract_text()
            pattern = r"Q\s*(\d+[A-Z]?\d*\)?)\.?\s+(.*)" 
            matches = re.findall(pattern, texte_page, re.MULTILINE)
            
            for match in matches:
                if match:
                    numero_question, titre = match
                    resultats.append((numero_question.strip()))

        return resultats


In [15]:
def get_nom_epreuve(file: str) -> str:
    """Donne le nom de l'épreuve à partir du fichier PDF.

    :param file: Lien du fichier PDF
    :type file: str
    :return: Nom de l'épreuve
    :rtype: str
    """
    page = extract_first_page_from_pdf(file)
    page_couper = page.split("\n")

    for ligne in page_couper:
        ligne = ligne.strip()

        # Ignorer les lignes contenant une date (français, anglais, portugais, format ISO, etc.)
        if re.search(r"(?:\w{3,9} \d{1,2},? \d{4}|\d{4}-\d{2}-\d{2}).*Page ?\d+/\d+", ligne, re.IGNORECASE):
            continue

        # Ignorer les codes du type "S013/2021-11-24"
        if re.match(r"[A-Z]?\d{3,4}/\d{4}-\d{2}-\d{2}", ligne):
            continue
        
        if ligne == " .. " or ligne == "..":
            continue

        # Séparer une année collée au titre (ex: "2011Portugais" → "Portugais")
        ligne = re.sub(r"^\d{4}\s*\.*\s*", "", ligne).strip()

        # Vérifier si la ligne peut être un titre (pas vide, ne commence pas par un chiffre, etc.)
        if ligne and not ligne[0].isdigit() and len(ligne.split()) <= 3:
            return ligne  # On retourne le premier titre valide trouvé

    return ""  # Retourne une chaîne vide si aucun titre n'est trouvé


In [16]:
def get_all_epreuve_name(folder:str)-> set[str]:
    """Renvoie tout les noms d'épreuves

    :param folder: chemin du dossier
    :type folder: str
    :return: tout les noms d'épreuves
    :rtype: set[str]
    """

    if os.listdir(folder) != []:
        liste_results = [extract_first_page_from_pdf(file) for file in link_all_files(folder)]
        liste_cut_results = [result.split("\n") for result in liste_results]
        liste_nom = [result[1] for result in liste_cut_results]
        
        # On enelve l'année devant la matiere
        liste_nv_deb  = [re.sub(r"\d*.? ?","" ,nom[:4]) for nom in liste_nom]
        liste_nv_nom = [deb+nom[4:] for deb,nom in zip(liste_nv_deb,liste_nom)]
        
        for i in range(len(liste_nv_nom)):
            if liste_nv_nom[i] == '..':
                liste_nom[i] = liste_results[i].split("\n")[2]
                liste_nv_deb[i] = re.sub(r"\d\.*\s*.*","" ,liste_nom[i][:4])
                liste_nv_nom[i] = liste_nv_deb[i]+liste_nom[i][4:]
        
        d:set = set()           
        for nom in liste_nv_nom:
            d.add(nom)
            
    return d


In [17]:
#all_epreuve = get_all_epreuve_name(r"data/centrale")

In [18]:
def initialise_liste_epreuve():
    """Parcours toutes les filières du dossier centrale et initialise la liste des epreuves
    Elle met donc a jour la base de donnee database.db
    """
    list_epreuve = []
    for f in filieres:
        all_epreuve = get_all_epreuve_name(r"data/centrale/"+f)
        for epreuve in all_epreuve:
            list_epreuve.append(gd.Epreuve(epreuve, f))

In [19]:
def rename_epreuve(lien_epreuve:str, nv_nom:str) -> None:
    """Renomme une epreuve

    :param lien_epreuve: lien vers le fichier de l'epreuve
    :type lien_epreuve: str
    :param nv_nom: le nouveau nom de l'epreuve
    :type nv_nom: str
    """
    try :os.rename(lien_epreuve, nv_nom)
    except:
        print("Erreur lors du renommage ", lien_epreuve, nv_nom)

In [37]:
SEUIL_SIMILARITE = 0.10  # Ajustable selon le besoin

In [ ]:
def rename_all_epreuve(dossier: str = "data/centrale") -> None:
    """Renomme toutes les épreuves du dossier par leur nom correct."""
    
    for f in filieres:
        if f not in liste_matiere:
            liste_matiere[f] = set()
        if f not in liste_epreuve:
            liste_epreuve[f] = {}

        for a in annee:
            liste_lien = link_all_files(f"{dossier}/{f}/{a}/")
            liste_nom = [get_nom_epreuve(file) for file in liste_lien]

            # Renommage des fichiers
            for i in range(len(liste_nom)):
                ancien_nom = liste_lien[i]
                nouveau_nom = f"{dossier}/{f}/{a}/{liste_nom[i]}.pdf"
                try:
                    rename_epreuve(ancien_nom, nouveau_nom)
                except Exception as e:
                    print(f"Erreur lors du renommage : {ancien_nom} -> {nouveau_nom}")
                    print(e)


In [22]:
mot1 = "Russe"
mot2 = "Physique"
Levenshtein.distance(mot1, mot2)/(len(mot1)+len(mot2))

0.46153846153846156

In [23]:
liste_epreuve = {f: {} for f in filieres}

In [ ]:
def get_fichiers_par_filiere(dossier: str) -> dict[str, list[set[str]]]:
    """Récupère les noms des épreuves par filière.

    :param dossier: dossier contenant les fichiers
    :type dossier: str
    :return:    dictionnaire des fichiers par filiere avec comme :
                - clef le nom de la filiere
                - valeur une liste de set de fichier
    :rtype: dict[str, list[set[str]]]
    """
    fichiers_par_filiere = {}
    for filiere in os.listdir(dossier):
        path_filiere = os.path.join(dossier, filiere)
        if os.path.isdir(path_filiere):
            fichiers = []
            for annee in os.listdir(path_filiere):
                path_annee = os.path.join(path_filiere, annee)
                if os.path.isdir(path_annee):
                    fichiers += [re.sub(r"\.pdf$", "", f) for f in os.listdir(path_annee) if f.endswith(".pdf")] #recupere les fichiers de type .pdf
            fichiers_par_filiere[filiere] = list(set(fichiers))
    return fichiers_par_filiere


In [39]:
def nettoyer_nom_matiere(nom):
    """Supprime les chiffres du nom de la matière."""
    return re.sub(r"?(\d+?)", "", nom).strip()

In [40]:
def trouver_meilleur_k(noms_epreuves):
    """Trouve le nombre optimal de clusters avec la silhouette score."""
    if len(noms_epreuves) < 2:
        return 1  # Pas de clustering possible
    
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(noms_epreuves)
    
    meilleurs_k = 1
    meilleur_score = -1
    for k in range(2, min(10, len(noms_epreuves)) + 1):
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        if score > meilleur_score:
            meilleur_score = score
            meilleurs_k = k
    
    return meilleurs_k

In [41]:
def normaliser_nom(nom):
    """Supprime les chiffres, met en minuscule et normalise le texte pour éviter les variantes d'écriture."""
    nom = re.sub(r"\(\d+\)|\d+", "", nom).strip().upper()
    nom = unicodedata.normalize("NFKD", nom)  # Supprime les accents
    return nom

In [42]:
def regrouper_par_distance_levenshtein(matieres):
    """Regroupe les matières ayant une faible distance de Levenshtein."""
    seuil_distance = 2  # Seuil pour considérer deux matières comme similaires
    matieres2 = copy.deepcopy(matieres)
    for matiere in matieres:
        for matiere2 in matieres:
            if matiere != matiere2 and Levenshtein.distance(matiere, matiere2) < seuil_distance:
                if len(matiere) > len(matiere2):
                    matieres2 - {matiere2}
                else: matieres2 = matieres2 - {matiere}
                 
    return matieres2

In [ ]:
def regrouper_matieres(fichiers_par_filiere: dict[str, list[set[str]]]) -> dict[str, set[str]]:
    """Regroupe les matières en appliquant un clustering sur les noms normalisés des épreuves.

    :param fichiers_par_filiere: Dictionnaire des fichiers par filière
    :type fichiers_par_filiere: dict[str, list[set[str]]]
    :return: Dictionnaire des matières par filière
    :rtype: dict[str, set[str]]
    """
    matieres_par_filiere = {}

    for filiere, noms_epreuves in fichiers_par_filiere.items():
        if not noms_epreuves:
            continue
        
        # Nettoyage et normalisation des noms
        noms_epreuves_nettoyes = [normaliser_nom(nom) for nom in noms_epreuves]
        
        # Trouver le meilleur k
        k = trouver_meilleur_k(noms_epreuves_nettoyes)
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(noms_epreuves_nettoyes)
        
        # Appliquer le clustering
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X)
        
        # Regrouper les matières selon les clusters
        clusters = {}
        for nom, label in zip(noms_epreuves_nettoyes, labels):
            if label not in clusters:
                clusters[label] = []
            clusters[label].append(nom)
        
        # Prendre le nom le plus fréquent comme représentant du cluster
        matieres_par_filiere[filiere] = {max(set(group), key=group.count) for group in clusters.values()}
        matieres_par_filiere[filiere] = regrouper_par_distance_levenshtein(matieres_par_filiere[filiere])
    
    return matieres_par_filiere

In [ ]:
def enregistrer_matieres(matieres_par_filiere):
    """Enregistre les matières regroupées dans la base de données."""
    connexion = sqlite3.connect("database.db")
    cursor = connexion.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS matieres (filiere TEXT, matiere TEXT, banque TEXT ,PRIMARY KEY (filiere, banque,matiere))")
    
    for filiere, matieres in matieres_par_filiere.items():
        for matiere in matieres:
            cursor.execute("INSERT OR IGNORE INTO matieres (filiere, banque ,matiere) VALUES (?, ? ,?)", (filiere, "Centrale",matiere))
    
    connexion.commit()
    connexion.close()

In [ ]:
# Exemple d'utilisation
dossier = "data/centrale"
fichiers_par_filiere = get_fichiers_par_filiere(dossier)
matieres_regroupees = regrouper_matieres(fichiers_par_filiere)
enregistrer_matieres(matieres_regroupees)

for filiere, matieres in matieres_regroupees.items():
    print(f"Filière {filiere}, Matières regroupées: {matieres}")

In [25]:
def matiere_of_epreuve(epreuve:str,filiere:str) -> str:
    """Renvoie la matiere de l'epreuve

    :param epreuve: le nom de l'epreuve
    :type epreuve: str
    :return: la matiere de l'epreuve
    :rtype: str
    """
    connexion = sqlite3.connect("database.db")
    cursor = connexion.cursor()
    cursor.execute("SELECT matiere FROM matieres WHERE filiere = ?" , (filiere,))
    matieres = cursor.fetchall()
    connexion.close()
    return min(matieres, key=lambda x: Levenshtein.distance(x[0], epreuve))[0]

In [26]:
def attribut_matiere_epreuve(filiere: str) -> None:
    """Attribue la matière à chaque épreuve de la filière.

    :param filiere: Nom de la filière.
    :type filiere: str
    """
    connexion = sqlite3.connect("database.db")
    cursor = connexion.cursor()  
    liste_lien = link_all_files(f"data/centrale/{filiere}")

    for lien in liste_lien:
        nom = get_nom_epreuve(lien)
        reponse = cursor.execute("SELECT * FROM epreuves WHERE nom = ? AND filiere = ?", (nom, filiere)).fetchone()

        if reponse is None:
            matiere = matiere_of_epreuve(nom, filiere)
            cursor.execute(
                "INSERT INTO epreuves (nom, filiere, matiere, banque ,nombre) VALUES (?, ?, ?, ?, ?)",
                (nom, filiere, "Centrale",matiere, 1)
            )
        else:
            cursor.execute(
                "UPDATE epreuves SET nombre = ? WHERE nom = ?", 
                (reponse[5] + 1, nom)
            )

    connexion.commit()
    connexion.close()


In [27]:
def association_matiere_epreuve() -> None:
    """Remplis la base de données 
    """
    for f in filieres:   
        attribut_matiere_epreuve(f)


In [28]:
association_matiere_epreuve()

In [ ]:
def recupere_info_sujet(file): 
    nom = get_nom_epreuve(file)
    annee = os.path.basename(os.path.dirname(file))
    filiere = os.path.basename(os.path.dirname(os.path.dirname(file)))
    matiere = matiere_of_epreuve(nom, filiere)
    return nom, annee, filiere, matiere
     

In [30]:
def creer_table_sujet():
    connexion = sqlite3.connect("database.db")
    cursor = connexion.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS sujets (id INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT, annee TEXT, filiere TEXT, matiere TEXT, banque TEXT)")
    connexion.commit()
    connexion.close()
    
    for lien in link_all_files("data/centrale"):
        nom, annee, filiere, matiere = recupere_info_sujet(lien)
        connexion = sqlite3.connect("database.db")
        cursor = connexion.cursor()
        cursor.execute("INSERT INTO sujets (nom, annee, filiere, matiere, banque) VALUES (?, ?, ?, ?, ?)", (nom, annee, filiere, matiere, "Centrale"))
        connexion.commit()
        connexion.close()
        

In [31]:
creer_table_sujet()

Allemand 2011 MP
Anglais 2011 MP
Arabe 2011 MP
Chinois 2011 MP
Espagnol 2011 MP
Informatique 2011 MP
Italien 2011 MP
Mathématiques 2 2011 MP
Mathématiques I 2011 MP
Physique Chimie 2011 MP
Physique 2011 MP
Physique–Chimie 2011 MP
Portugais 2011 MP
Russe 2011 MP
Rédaction 2011 MP
S2I 2011 MP
Allemand 2012 MP
Anglais 2012 MP
Arabe 2012 MP
Chinois 2012 MP
Espagnol 2012 MP
Informatique 2012 MP
Italien 2012 MP
Mathématiques 1 2012 MP
Mathématiques 2 2012 MP
Physique 2012 MP
Physique–Chimie 2012 MP
Portugais 2012 MP
Russe 2012 MP
Rédaction 2012 MP
S2I 2012 MP
Allemand 2013 MP
Anglais 2013 MP
Arabe 2013 MP
Chinois 2013 MP
Espagnol 2013 MP
Informatique 2013 MP
Italien 2013 MP
Mathématiques 1 2013 MP
Mathématiques 2 2013 MP
Physique 2013 MP
Physique–Chimie 2013 MP
Portugais 2013 MP
Russe 2013 MP
Rédaction 2013 MP
S2I 2013 MP
Allemand 2014 MP
Anglais 2014 MP
Arabe 2014 MP
Chinois 2014 MP
Espagnol 2014 MP
Informatique 2014 MP
Italien 2014 MP
Mathématiques 1 2014 MP
Mathématiques 2 2014 MP
Physiqu

# Fonction générique
## Traitement des sujets

In [64]:
def enleve_nombre(nom: str) -> str:
    """
    Supprime les nombres et les tirets qui les precedent d'un nom.

    :param nom: Nom a nettoyer
    :return: Nom nettoye
    """
    return re.sub(r'[-]?\d+', '', nom).strip()

In [48]:
def compute_distance_matrix(words):
    """Calcule la matrice des distances de Levenshtein entre les mots."""
    n = len(words)
    distance_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i + 1, n):
            dist = levenshtein_distance(words[i], words[j])
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist  # Matrice symétrique

    return distance_matrix

In [49]:
def optimal_k_selection(distance_matrix, max_k=10):
    """Trouve le meilleur K en utilisant le coefficient de silhouette."""
    best_k = 2
    best_score = -1
    for k in range(2, min(max_k, len(distance_matrix))):  # Éviter k > n
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(distance_matrix)
        score = silhouette_score(distance_matrix, labels, metric="precomputed")
        
        if score > best_score:
            best_k = k
            best_score = score

    return best_k

In [50]:
def kmeans_clustering(words, max_k=10):
    """Effectue le clustering des mots en utilisant K-Means avec la distance de Levenshtein."""
    if len(words) < 2:
        return {0: words}  # Un seul mot → un seul cluster
    
    distance_matrix = compute_distance_matrix(words)
    best_k = optimal_k_selection(distance_matrix, max_k)

    kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(distance_matrix)

    clusters = {}
    for i, label in enumerate(labels):
        clusters.setdefault(label, []).append(words[i])

    return clusters


## SQL Générique

In [ ]:
def ajout_epreuve(nom:str,filiere:str,matiere:str,banque:str):
    """Rajoute une epreuve dans la base de données

    :param nom: nom de l'epreuve
    :type nom: str
    :param filiere: nom de la filiere (ex : MP,PC,PSI)
    :type filiere: str
    :param matiere: nom de la matiere
    :type matiere: str
    :param banque: nom de la banque
    :type banque: str
    """
    connexion = sqlite3.connect("database.db")
    cursor = connexion.cursor()
    cursor.execute("INSERT INTO epreuves (nom, filiere, matiere, banque ,nombre) VALUES (?, ?, ?, ?, ?)", (nom, filiere, matiere, banque, 1))
    connexion.commit()
    connexion.close()

# Mines

In [9]:
lien_mines = "https://wordpress.concoursminesponts.fr/wp-content/uploads/2024/07/"

In [10]:
def extract_zip_mines(lien_mines:str = lien_mines) -> list[str]:
    """Extrait les liens finissant par .zip d'une page web

    :param lien_mines: lien vers le site internet, defaults to lien_mines
    :type lien_mines: str, optional
    :return: liste des noms de fichiers zip
    :rtype: list[str]
    """
    links = extract_links_from_page(lien_mines)
    liste_lien = []
    for link in links:
        if link.endswith(".zip"):
            liste_lien.append(link)
    return liste_lien

In [11]:
def name_of_zip(lien:str) -> str:
    """
    Renvoie le nom du fichier sans le .zip

    :param lien: lien internet vers le fichier .zip
    :type lien: str
    :return: nom du zip
    :rtype: str
    """
    return os.path.basename(lien).replace(".zip","")

In [12]:
def unzip_mines(lien:str)-> None:
    """Telecharge les fichiers zip et les extrait

    :param lien: lien internet vers le fichier .zip
    :type lien: str
    """
    req = requests.get(lien,stream=True)
    nom = name_of_zip(lien)
    lien_base = "stockage_zip"
    lien_stockage_pack = lien_base+"/zip_pack/"+nom+".zip" 
    lien_stockage_unpack = lien_base+"/zip_unpack" 
    try:
        urllib.request.urlretrieve(lien, lien_stockage_pack)
        #  Extrait le ZIP
        with ZipFile(lien_stockage_pack, 'r') as zip_ref:
            zip_ref.extractall(lien_stockage_unpack)
            
    except FileNotFoundError:
        if not os.path.exists(lien_base):
            os.mkdir(lien_base)
            os.mkdir(lien_base+"/zip_pack/")
            os.mkdir(lien_base+"/zip_unpack" )
            unzip_mines(lien)
        elif not os.path.exists(lien_base+"/zip_pack/"):
            os.mkdir(lien_base+"/zip_pack/")
            unzip_mines(lien)
        elif not os.path.exists(lien_base+"/zip_unpack" ):
            os.mkdir(lien_base)
            unzip_mines(lien)
        else:
            print("Erreur lors de l'extraction du fichier")        


In [13]:
def recuperer_mines(lien_mines:str = lien_mines) -> None:
    """Recuperer tous les fichiers zip, les telecharges et les extraits 

    :param lien_mines: lien de la page internet, defaults to lien_mines
    :type lien_mines: str, optional
    """
    liste_lien = extract_zip_mines(lien_mines)
    for lien in liste_lien:
        unzip_mines(lien)

In [14]:
def recupere_lien_mines_atraiter() -> list[str]:
    try : 
        lien = "stockage_zip/zip_unpack"
        liste_lien_complete = os.listdir(lien)
        liste_lien = []
        for lien in liste_lien_complete:
            if any(c.isdigit() for c in lien):
                liste_lien.append("stockage_zip/zip_unpack/"+lien)
        return liste_lien
    except FileNotFoundError:
        print("Le dossier " + lien + " n'existe pas")

In [ ]:
def get_all_sujet_mines() -> dict[str, dict[str, list[str]]]:
    """Parcours tous les sujets mines

    :return: Dictionnaire ayant comme clef les annee et comme valeur un dictionnaire ayant comme clef les matieres ou les filieres et comme valeur la liste des noms des sujets 
    :rtype: dict[str, dict[str, list[str]]]
    """
    liste_lien = recupere_lien_mines_atraiter()
    reponse = {}
    for annee in liste_lien:
        nom_annee = os.path.basename(annee)
        filiers = [lien for lien in os.listdir(annee) if lien != ".DS_Store"]
        n_annee = "".join(re.findall(r"\d+", os.path.basename(annee)))   #recupere le nom du sujet et en prend uniquement les chiffres
        reponse[n_annee] = {}
        for f in filiers:
            if os.path.isdir(annee+"/"+ f):
                reponse_annee_filiere = [sujet for sujet in os.listdir(annee+"/"+ f) if os.path.isdir(annee+"/"+ f+"/"+sujet) or (annee+"/"+ f+"/"+sujet).lower().endswith(".pdf")]
                reponse[n_annee][f] = reponse_annee_filiere
            else: 
                if "EPREUVES-COMMUNES" in reponse[n_annee]:
                    reponse[n_annee]["EPREUVES-COMMUNES"].append(f)
                else:
                    reponse[n_annee]["EPREUVES-COMMUNES"] = [f]
    return reponse

In [ ]:
def get_type_nom(nom_dossier):

In [ ]:
def get_filiere_of_sujet_mines(nom_dossier:str) -> str:
    """Renvoie la filiere a partir du nom du dossier si possible

    :param nom_dossier: nom du dossier
    :type nom_dossier: str
    :return: filiere du dossier
    :rtype: str
    """
    

In [21]:
all_sujet = get_all_sujet_mines()

In [31]:
nom_key = [all_sujet[annee].keys() for annee in all_sujet]
liste_key = []
for liste in nom_key:
    for key in liste:
        liste_key.append(key)
print(liste_key)

['EPREUVES-COMMUNES', 'MP-2014', 'PC-2014', 'PSI-2014', 'EPREUVES-COMMUNES', 'MP-2015', 'PC-2015', 'PSI-2015', 'EPREUVES-COMMUNES', 'MP-2016', 'PC-2016', 'PSI-2016', 'EPREUVES-COMMUNES', 'MP-2017', 'PC-2017', 'PSI-2017', 'EPREUVES-COMMUNES-2018', 'MP-2018', 'PC-2018', 'PSI-2018', 'EPREUVES-COMMUNES-2019', 'MP-2019', 'PC-2019', 'PSI-2019', 'EPREUVES-COMMUNES', 'MP', 'PC', 'PSI', 'EPREUVES-COMMUNES', 'MP', 'PC', 'PSI', 'EPREUVES-COMMUNES', 'MP', 'PC', 'PSI', 'Chimie', 'EPREUVES-COMMUNES', 'Informatique', 'Maths', 'Physique', 'SI', 'EPREUVES-COMMUNES', 'SI']


In [ ]:
# nom du dossier
os.path.basename("stockage_zip/zip_unpack/2024/MP")

'MP'

In [30]:
def detect_type(keys_list):
    """Classifie dynamiquement les groupes en filières ou matières en fonction de la longueur des mots."""
    
    classification = {"filieres": [], "matieres": []}
    
    for keys in keys_list:
        mots = set(keys)  # Convertir en set pour éviter les doublons

        # On sépare selon la longueur des mots
        filieres = {mot for mot in mots if len(mot) <= 5}  # Mots courts -> filières
        matieres = mots - filieres  # Mots longs -> matières

        if len(filieres) > 1:  # Plusieurs filières ensemble -> c'est bien un groupe de filières
            classification["filieres"].append(keys)
        else:
            classification["matieres"].append(keys)

    return classification

In [66]:
liste_key = [enleve_nombre(key) for key in liste_key]
kmeans_clustering(liste_key)

{0: ['EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES',
  'EPREUVES-COMMUNES'],
 4: ['MP', 'MP', 'MP', 'MP', 'MP', 'MP', 'MP', 'MP', 'MP'],
 5: ['PC', 'PC', 'PC', 'PC', 'PC', 'PC', 'PC', 'PC', 'PC'],
 6: ['PSI', 'PSI', 'PSI', 'PSI', 'PSI', 'PSI', 'PSI', 'PSI', 'PSI'],
 8: ['Chimie'],
 3: ['Informatique'],
 7: ['Maths'],
 2: ['Physique'],
 1: ['SI', 'SI']}